In [ ]:
pip install Datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 2.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from datasets import Dataset, DatasetDict
import transformers
from transformers import AutoTokenizer
import torch
import torch.nn.functional as F
from transformers import AutoModel
from sklearn.metrics import accuracy_score, f1_score
from sklearn.linear_model import LogisticRegression

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
trainData = pd.read_csv('/content/drive/MyDrive/NLP/train.csv', encoding='ISO-8859-1') # file encoding is not UTF-8, which is the pandas default encoding when reading csv, so we had specified the encoding
trainData = trainData.dropna()
print(trainData.head())

       textID                                               text  \
0  cb774db0d1                I`d have responded, if I were going   
1  549e992a42      Sooo SAD I will miss you here in San Diego!!!   
2  088c60f138                          my boss is bullying me...   
3  9642c003ef                     what interview! leave me alone   
4  358bd9e861   Sons of ****, why couldn`t they put them on t...   

                         selected_text sentiment Time of Tweet Age of User  \
0  I`d have responded, if I were going   neutral       morning        0-20   
1                             Sooo SAD  negative          noon       21-30   
2                          bullying me  negative         night       31-45   
3                       leave me alone  negative       morning       46-60   
4                        Sons of ****,  negative          noon       60-70   

       Country  Population -2020  Land Area (Km²)  Density (P/Km²)  
0  Afghanistan          38928346         652860.0    

In [ ]:
columns_to_drop = ['textID', 'selected_text', 'Time of Tweet', 'Age of User', 'Country', 'Population -2020', 'Land Area (Km²)', 'Density (P/Km²)']
trainData = trainData.drop(columns=columns_to_drop)
print("DataFrame after dropping columns:")
print(trainData.head())

DataFrame after dropping columns:
                                                text sentiment
0                I`d have responded, if I were going   neutral
1      Sooo SAD I will miss you here in San Diego!!!  negative
2                          my boss is bullying me...  negative
3                     what interview! leave me alone  negative
4   Sons of ****, why couldn`t they put them on t...  negative


In [ ]:
testData = pd.read_csv('/content/drive/MyDrive/NLP/test.csv', encoding='ISO-8859-1') # file encoding is not UTF-8, which is the pandas default encoding when reading csv, so we had specified the encoding
testData = testData.dropna()
print(testData.head())

       textID                                               text sentiment  \
0  f87dea47db  Last session of the day  http://twitpic.com/67ezh   neutral   
1  96d74cb729   Shanghai is also really exciting (precisely -...  positive   
2  eee518ae67  Recession hit Veronique Branquinho, she has to...  negative   
3  01082688c6                                        happy bday!  positive   
4  33987a8ee5             http://twitpic.com/4w75p - I like it!!  positive   

  Time of Tweet Age of User      Country  Population -2020  Land Area (Km²)  \
0       morning        0-20  Afghanistan        38928346.0         652860.0   
1          noon       21-30      Albania         2877797.0          27400.0   
2         night       31-45      Algeria        43851044.0        2381740.0   
3       morning       46-60      Andorra           77265.0            470.0   
4          noon       60-70       Angola        32866272.0        1246700.0   

   Density (P/Km²)  
0             60.0  
1            1

In [ ]:
columns_to_drop = ['textID', 'Time of Tweet', 'Age of User', 'Country', 'Population -2020', 'Land Area (Km²)', 'Density (P/Km²)']
testData = testData.drop(columns=columns_to_drop)
print("DataFrame after dropping columns:")
print(testData.head())

DataFrame after dropping columns:
                                                text sentiment
0  Last session of the day  http://twitpic.com/67ezh   neutral
1   Shanghai is also really exciting (precisely -...  positive
2  Recession hit Veronique Branquinho, she has to...  negative
3                                        happy bday!  positive
4             http://twitpic.com/4w75p - I like it!!  positive


In [ ]:
# Now, we are going to split the trainData in train set and validation set.
train_data, val_data = train_test_split(trainData, test_size=0.2, random_state=42) # 20% of the train set is going to be the validation set

print("Training set shape:", train_data.shape)
print("Validation set shape:", val_data.shape)
print("Test set shape:", testData.shape)
# For instance now, we have train_data, val_data and testData datasets.

Training set shape: (21984, 2)
Validation set shape: (5496, 2)
Test set shape: (3534, 2)


In [ ]:
train_data = Dataset.from_pandas(train_data) # 'dataset.from_pandas' is a function from HuggingFace to convert the dataset into a HuggingFace Dataset format
val_data = Dataset.from_pandas(val_data)
testData = Dataset.from_pandas(testData)

train_data = train_data.class_encode_column('sentiment') #call_encode_column --> encodes the 3 categories from 'sentiment'
val_data = val_data.class_encode_column('sentiment')
test_data = testData.class_encode_column('sentiment')

dataset_dict = DatasetDict({
    'train': train_data,
    'eval': val_data,
    'test': test_data
})

print(dataset_dict)
print(dataset_dict['train'].features)

Casting to class labels:   0%|          | 0/21984 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/5496 [00:00<?, ? examples/s]

Casting to class labels:   0%|          | 0/3534 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'sentiment', '__index_level_0__'],
        num_rows: 21984
    })
    eval: Dataset({
        features: ['text', 'sentiment', '__index_level_0__'],
        num_rows: 5496
    })
    test: Dataset({
        features: ['text', 'sentiment', '__index_level_0__'],
        num_rows: 3534
    })
})
{'text': Value(dtype='string', id=None), 'sentiment': ClassLabel(names=['negative', 'neutral', 'positive'], id=None), '__index_level_0__': Value(dtype='int64', id=None)}


In [ ]:
print(type(train_data)) # to check if it is now in a HuggingFace dataset format

<class 'datasets.arrow_dataset.Dataset'>


## Bert-base model with 'text' column

In [ ]:
model_ckpt_base = "bert-base-uncased" # download bert-base model
tokenizer_base = AutoTokenizer.from_pretrained(model_ckpt_base)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def tokenize(batch):
    return tokenizer_base(batch["text"], padding=True, truncation=True, return_tensors="pt")

In [ ]:
# ensure that the dataset is prepared to work with PyTorch tensors:
dataset_dict.set_format(type="torch")

# applies the tokenize function to the dataset_dict to transform each example in the dataset (to tokens_ids)
tweets_encoded = dataset_dict.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/21984 [00:00<?, ? examples/s]

Map:   0%|          | 0/5496 [00:00<?, ? examples/s]

Map:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [ ]:
print(tweets_encoded['train'].column_names)

['text', 'sentiment', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask']


In [ ]:
# Load the entire model of BERT:
model_ckpt = "bert-base-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def extract_hidden_states(batch):
    # Place model inputs on the GPU
    inputs = {k:v.to(device) for k,v in batch.items() # places the model inputs into de GPU
              if k in tokenizer.model_input_names}
    # Extract last hidden states
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state # feeds the inputs into the model
    # Return vector for [CLS] token
    return {"hidden_state": last_hidden_state[:,0].cpu().numpy()} # extracts the hidden state corresponding to the [CLS] token, which is located at index 0 of the second dimension of the last_hidden_state tensor

In [ ]:
tweets_encoded.set_format("torch", columns=["input_ids", "attention_mask", "sentiment"])

#hide_output
tweets_hidden = tweets_encoded.map(extract_hidden_states, batched=True)

Map:   0%|          | 0/21984 [00:00<?, ? examples/s]

KeyboardInterrupt: 

In [ ]:
torch.save(tweets_hidden, 'tweets_hidden_BERTbase.pt')

In [ ]:
tweets_hidden = torch.load('/content/drive/MyDrive/NLP/tweets_hidden_BERTbase.pt') # Load again since it takes almost an hour to run

<ipython-input-18-3ce2ca6af83f>:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tweets_hidden = torch.load('/content/drive/MyDrive/NLP/tweets_hidden_BERTbase.pt') # Load ag

In [ ]:
tweets_hidden["train"].column_names

['text',
 'sentiment',
 '__index_level_0__',
 'input_ids',
 'token_type_ids',
 'attention_mask',
 'hidden_state']

In [ ]:
X_train = np.array(tweets_hidden["train"]["hidden_state"])
X_valid = np.array(tweets_hidden["eval"]["hidden_state"])
X_test = np.array(tweets_hidden["test"]["hidden_state"])
y_train = np.array(tweets_hidden["train"]["sentiment"])
y_valid = np.array(tweets_hidden["eval"]["sentiment"])
y_test = np.array(tweets_hidden["test"]["sentiment"])
X_train.shape, X_valid.shape, X_test.shape

((21984, 768), (5496, 768), (3534, 768))

In [ ]:
# Test with LogisticRegression:
lr_clf = LogisticRegression(max_iter=3000)
lr_clf.fit(X_train, y_train)

LogisticRegression(max_iter=3000)

In [ ]:
print(f"Accuracy for eval set is {lr_clf.score(X_valid, y_valid)}")
print(f"Accuracy for test set is {lr_clf.score(X_test, y_test)}")

Accuracy for eval set is 0.6699417758369723
Accuracy for test set is 0.6703452178834183


#### Fine-tuning for BERT_base

In [ ]:
# hide_output
from transformers import AutoModelForSequenceClassification

num_labels = 3
model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels=num_labels)
         .to(device))

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

#### EarlyStopping

In [ ]:
from transformers import Trainer, TrainingArguments
batch_size = 64
logging_steps = len(tweets_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-tweets"

args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=25,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  evaluation_strategy="epoch",  # Use "epoch" for evaluation strategy
                                  save_strategy="epoch",
                                  weight_decay=0.01,
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False,
                                  log_level="error",
                                  report_to="none",
                                  load_best_model_at_end=True)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback, IntervalStrategy
tweets_encoded = tweets_encoded.rename_column("sentiment", "label")
trainer = Trainer(model=model, args=args,
                  compute_metrics=compute_metrics,
                  train_dataset=tweets_encoded["train"],
                  eval_dataset=tweets_encoded["eval"],
                  callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
                  tokenizer=tokenizer)

In [ ]:
trainer.train();

Epoch,Training Loss,Validation Loss,Accuracy,F1
1,0.617400,0.512165,0.787300,0.786952
2,0.441400,0.508403,0.801856,0.802111
3,0.331200,0.558703,0.797489,0.797248
4,0.235800,0.634314,0.786390,0.786311
5,0.161000,0.796210,0.782933,0.781834


#### Normal training:

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(tweets_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-news"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False,
                                  log_level="error")

In [ ]:
from transformers import Trainer
tweets_encoded = tweets_encoded.rename_column("sentiment", "label") #IMPORTANTE: CAMBIAR EL NOMBRE DE LA COLUMNA DONDE TENEMOS EL LABEL A LABEL XQ SINO NO FUNCIONA
trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=tweets_encoded["train"],
                  eval_dataset=tweets_encoded["eval"],
                  tokenizer=tokenizer)
trainer.train();

## Bert-large model with 'text' column

In [ ]:
model_ckpt_large = "bert-large-uncased"
tokenizer_large = AutoTokenizer.from_pretrained(model_ckpt_large)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
def tokenize(batch):
    return tokenizer_large(batch["text"], padding=True, truncation=True, return_tensors="pt")

In [ ]:
# ensure that the dataset is prepared to work with PyTorch tensors:
dataset_dict.set_format(type="torch")

# applies the tokenize function to the dataset_dict to transform each example in the dataset (to tokens_ids)
tweets_encoded = dataset_dict.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/21984 [00:00<?, ? examples/s]

Map:   0%|          | 0/5496 [00:00<?, ? examples/s]

Map:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [ ]:
print(tweets_encoded['train'].column_names)

['text', 'sentiment', '__index_level_0__', 'input_ids', 'token_type_ids', 'attention_mask']


In [ ]:
model_ckpt = "bert-large-uncased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModel.from_pretrained(model_ckpt).to(device)

OutOfMemoryError: CUDA out of memory. Tried to allocate 120.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 109.06 MiB is free. Process 11129 has 14.64 GiB memory in use. Of the allocated memory 13.43 GiB is allocated by PyTorch, and 1.08 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)

In [ ]:
def extract_hidden_states(batch):
    # Place model inputs on the GPU
    inputs = {k:v.to(device) for k,v in batch.items() # places the model inputs into de GPU
              if k in tokenizer.model_input_names}
    # Extract last hidden states
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state # feeds the inputs into the model
    # Return vector for [CLS] token
    return {"hidden_state": last_hidden_state[:,0].cpu().numpy()} # extracts the hidden state corresponding to the [CLS] token, which is located at index 0 of the second dimension of the last_hidden_state tensor

In [ ]:
tweets_encoded.set_format("torch", columns=["input_ids", "attention_mask", "sentiment"])

#hide_output
tweets_hidden = tweets_encoded.map(extract_hidden_states, batched=True)

Map:   0%|          | 0/21984 [00:00<?, ? examples/s]

Map:   0%|          | 0/5496 [00:00<?, ? examples/s]

Map:   0%|          | 0/3534 [00:00<?, ? examples/s]

In [ ]:
torch.save(tweets_hidden, 'tweets_hidden_BERTlarge.pt')

In [ ]:
tweets_hidden = torch.load('/content/drive/MyDrive/NLP/tweets_hidden_BERTlarge.pt')

In [ ]:
X_train = np.array(tweets_hidden["train"]["hidden_state"])
X_valid = np.array(tweets_hidden["eval"]["hidden_state"])
X_test = np.array(tweets_hidden["test"]["hidden_state"])
y_train = np.array(tweets_hidden["train"]["sentiment"])
y_valid = np.array(tweets_hidden["eval"]["sentiment"])
y_test = np.array(tweets_hidden["test"]["sentiment"])
X_train.shape, X_valid.shape, X_test.shape

((21984, 1024), (5496, 1024), (3534, 1024))

In [ ]:
lr_clf = LogisticRegression(max_iter=3000)
lr_clf.fit(X_train, y_train)

LogisticRegression(max_iter=3000)

In [ ]:
print(f"Accuracy for eval set is {lr_clf.score(X_valid, y_valid)}")
print(f"Accuracy for test set is {lr_clf.score(X_test, y_test)}")

Accuracy for eval set is 0.6612081513828238
Accuracy for test set is 0.6522354272778721


#### Fine-tuning for BERT_large

In [ ]:
# hide_output
from transformers import AutoModelForSequenceClassification

num_labels = 3
model = (AutoModelForSequenceClassification
         .from_pretrained(model_ckpt, num_labels=num_labels)
         .to(device))

OutOfMemoryError: CUDA out of memory. Tried to allocate 120.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 109.06 MiB is free. Process 11129 has 14.64 GiB memory in use. Of the allocated memory 13.43 GiB is allocated by PyTorch, and 1.08 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

#### Early stopping:

In [ ]:
from transformers import Trainer, TrainingArguments
batch_size = 16
logging_steps = len(tweets_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-tweets"

args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=25,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  evaluation_strategy="epoch",  # Use "epoch" for evaluation strategy
                                  save_strategy="epoch",
                                  weight_decay=0.01,
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False,
                                  log_level="error",
                                  report_to="none",
                                  load_best_model_at_end=True)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Trainer, TrainingArguments
from transformers import EarlyStoppingCallback, IntervalStrategy
tweets_encoded = tweets_encoded.rename_column("sentiment", "label")
trainer = Trainer(model=model, args=args,
                  compute_metrics=compute_metrics,
                  train_dataset=tweets_encoded["train"],
                  eval_dataset=tweets_encoded["eval"],
                  callbacks = [EarlyStoppingCallback(early_stopping_patience=3)],
                  tokenizer=tokenizer)

In [ ]:
trainer.train();

Epoch,Training Loss,Validation Loss


OutOfMemoryError: CUDA out of memory. Tried to allocate 110.00 MiB. GPU 0 has a total capacity of 14.75 GiB of which 109.06 MiB is free. Process 11129 has 14.64 GiB memory in use. Of the allocated memory 13.43 GiB is allocated by PyTorch, and 1.08 GiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

#### Normal:

In [ ]:
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(tweets_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-news"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=25,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False,
                                  log_level="error")

In [ ]:
from transformers import Trainer
tweets_encoded = tweets_encoded.rename_column("sentiment", "label") #IMPORTANTE: CAMBIAR EL NOMBRE DE LA COLUMNA DONDE TENEMOS EL LABEL A LABEL XQ SINO NO FUNCIONA
trainer = Trainer(model=model, args=training_args,
                  compute_metrics=compute_metrics,
                  train_dataset=tweets_encoded["train"],
                  eval_dataset=tweets_encoded["eval"],
                  tokenizer=tokenizer)
trainer.train();